<h1 style="text-align: center;">Data Mining Project 3: Link Analysis</h1>

<p style="text-align:center;">
    呂伯駿<br>
    Q56074085<br>
    NetDB<br>
    National Cheng Kung University<br>
    pclu@netdb.csie.ncku.edu.tw
</p>


## 1. Introduction

本次報告的目的是觀察不同參數之 Graph 其 Hubs, authority, pageRank 與 Simrank 的變化，並透過增減 links 實驗了解參數對數值的影響。


## 2. Environment

這個部分我將會說明實驗所使用的環境與使用資料集。

### 2.1 System Preferences

實驗環境如下：
- <b>Operating System</b>: macOS High Mojave (10.14.1)
- <b>CPU</b>: 1.3 GHz Intel Core i5
- <b>Memory</b>: 8 GB 1600 MHz DDR3
- <b>Programming Language</b> : Python 3.6.2

由於所用之 Graph Data 最多僅有 5220 個Link，實驗環境記憶體足以符合實驗需求。

### 2.2 Dataset

#### 資料參數：

實驗使用 7 種 Graph，其 nodes 數目與 links 數目如下表：

| Graph ID   |     Nodes      |  Edges |
|----------|:-------------:|------:|
| 1 |  6 | 5  |
| 2 |  5 | 5  |
| 3 |  4 | 6  |
| 4 |  7 |  18 |
| 5 | 469  |  1102 |
| 6 |  1228 | 5220  |
| 7 |  2316 | 4714  |



## 3. Implementation

### 3.1 HITS




### 3.2 PageRank



### 3.3 SimRank



## 4. Analysis

### 4.1 Results


#### a. Graph 1




#### b. Graph 2



#### c. Graph 3


#### d. Graph 4

#### e. Graph 5

#### f. Graph 6


#### g. Graph 7



### 4.2 Methods to Increase hub, authority,  and PageRank

本小節嘗試透過增減 links 與 nodes 來增加 node 1 的 hub, authority, and PageRank。

#### a. Graph 1

此 graph 有 6 個 node，5 個 link，若不增減 node，則最多可再增加 25 條 link。

##### Hub
以 hub 為例，其 outlinks 且外連 node 的 authority 越高則 hub 越高。

若不斷增加 node 1 的 outlinks 數目，我們的確可以觀察到其 hub 數值隨之增加

Fig . 為增加 link 的結果，在原本的 graph 中，依序增加 '1,3', '1,4', '1,5', '1,6' 各 node 的 hub 數值變化


...

此外也可以觀察到若只增加 node 1 的 outlink，則其 hub 的數值僅與外連 node 的數目有關，而與其外連的對象無關。

從 Fig . 也可看到，在只增加 node 1 outlink 的情況下，其 authorities 並沒有變化，維持為 0。

##### Authority





##### PageRank


#### b. Graph 2


#### c. Graph 3





## 5. Conclusion


## 6. References

- [] Glen Jeh, Jennifer Widom. SimRank: A Measure of Structural-Context Similarity